In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
data = pd.read_excel('dataset.xlsx')
data.fillna(0, inplace=True)
data.isnull().sum()
data.head()

,From Date,To Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone
0,19-02-2022 00:00,20-02-2022 00:00,94.22,206.05,38.05,52.75,59.14,34.77,11.08,1.19,42.83
1,20-02-2022 00:00,21-02-2022 00:00,77.20,175.59,8.25,44.14,30.37,28.73,9.78,1.38,60.86
2,21-02-2022 00:00,22-02-2022 00:00,85.19,219.39,65.20,52.64,81.44,25.33,9.87,2.39,40.01
3,22-02-2022 00:00,23-02-2022 00:00,107.57,269.16,36.97,35.09,49.83,37.96,14.06,1.33,79.56
4,23-02-2022 00:00,24-02-2022 00:00,113.24,208.93,15.29,26.74,26.95,31.46,8.42,1.87,67.35


In [23]:
data['From Date'] = pd.to_datetime(data['From Date'])
data['To Date'] = pd.to_datetime(data['To Date'])
data.head()

C:\Users\Asus\AppData\Local\Temp\ipykernel_17688\2699852780.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['From Date'] = pd.to_datetime(data['From Date'])
C:\Users\Asus\AppData\Local\Temp\ipykernel_17688\2699852780.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['To Date'] = pd.to_datetime(data['To Date'])


,From Date,To Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone
0,2022-02-19,2022-02-20,94.22,206.05,38.05,52.75,59.14,34.77,11.08,1.19,42.83
1,2022-02-20,2022-02-21,77.20,175.59,8.25,44.14,30.37,28.73,9.78,1.38,60.86
2,2022-02-21,2022-02-22,85.19,219.39,65.20,52.64,81.44,25.33,9.87,2.39,40.01
3,2022-02-22,2022-02-23,107.57,269.16,36.97,35.09,49.83,37.96,14.06,1.33,79.56
4,2022-02-23,2022-02-24,113.24,208.93,15.29,26.74,26.95,31.46,8.42,1.87,67.35


In [24]:
df = data[['From Date', 'PM2.5']]
df.head()

,From Date,PM2.5
0,2022-02-19,94.22
1,2022-02-20,77.20
2,2022-02-21,85.19
3,2022-02-22,107.57
4,2022-02-23,113.24


In [25]:
# Train LLM(Large Language Model) model
text = ''
for i in range(0, len(df)):
    text = 'On ' + str(df['From Date'][i]) + ' the PM2.5 was ' + str(df['PM2.5'][i]) + '. ' + text
text

'On 2024-02-20 00:00:00 the PM2.5 was 78.32. On 2024-02-19 00:00:00 the PM2.5 was 76.97. On 2024-02-18 00:00:00 the PM2.5 was 108.45. On 2024-02-17 00:00:00 the PM2.5 was 90.57. On 2024-02-16 00:00:00 the PM2.5 was 87.31. On 2024-02-15 00:00:00 the PM2.5 was 103.17. On 2024-02-14 00:00:00 the PM2.5 was 147.55. On 2024-02-13 00:00:00 the PM2.5 was 166.64. On 2024-02-12 00:00:00 the PM2.5 was 139.09. On 2024-02-11 00:00:00 the PM2.5 was 159.31. On 2024-02-10 00:00:00 the PM2.5 was 201.33. On 2024-02-09 00:00:00 the PM2.5 was 73.26. On 2024-02-08 00:00:00 the PM2.5 was 63.72. On 2024-02-07 00:00:00 the PM2.5 was 65.8. On 2024-02-06 00:00:00 the PM2.5 was 62.58. On 2024-02-05 00:00:00 the PM2.5 was 62.94. On 2024-02-04 00:00:00 the PM2.5 was 104.56. On 2024-02-03 00:00:00 the PM2.5 was 84.04. On 2024-02-02 00:00:00 the PM2.5 was 76.36. On 2024-02-01 00:00:00 the PM2.5 was 29.05. On 2024-01-31 00:00:00 the PM2.5 was 187.43. On 2024-01-30 00:00:00 the PM2.5 was 277.11. On 2024-01-29 00:00:00

#### Using bert model to predict the predictions.

In [26]:
# Tokenizer and model training
from transformers import BertConfig, BertTokenizer, BertForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig()
model = BertForMaskedLM(config)

In [27]:
import torch
chunks_size = 100
chunks = [text[i:i+chunks_size] for i in range(0, len(text), chunks_size)]
tokens = tokenizer.encode(chunks[0], return_tensors='pt')
for chunk in chunks[1:]:
    tokens = tokenizer.encode(chunk, return_tensors='pt')
    tokens = torch.cat([tokens, tokenizer.encode(chunk, return_tensors='pt')], dim=-1)
tokens

tensor([[  101,  6185,  2475,  1011,  6185,  1011,  2322,  4002,  1024,  4002,
          1024,  4002,  1996,  7610,  2475,  1012,  1019,  2001,  6255,  1012,
          1016,  1012,  2006, 16798,  2475,  1011,  6185,  1011,  2539,  4002,
          1024,  4002,  1024,  4002,  1996,  7610,  2475,  1012,  1019,  2001,
          6365,  1012,  2570,  1012,   102,   101,  6185,  2475,  1011,  6185,
          1011,  2322,  4002,  1024,  4002,  1024,  4002,  1996,  7610,  2475,
          1012,  1019,  2001,  6255,  1012,  1016,  1012,  2006, 16798,  2475,
          1011,  6185,  1011,  2539,  4002,  1024,  4002,  1024,  4002,  1996,
          7610,  2475,  1012,  1019,  2001,  6365,  1012,  2570,  1012,   102]])

In [28]:
# Save text
with open('dataset.txt', 'w') as f:
    f.write(text)

In [29]:
# Train model on our data
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='dataset.txt',
    block_size=128,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)


c:\Users\Asus\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [30]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

c:\Users\Asus\anaconda3\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

{'train_runtime': 3.1158, 'train_samples_per_second': 0.321, 'train_steps_per_second': 0.321, 'train_loss': 10.491649627685547, 'epoch': 1.0}


TrainOutput(global_step=1, training_loss=10.491649627685547, metrics={'train_runtime': 3.1158, 'train_samples_per_second': 0.321, 'train_steps_per_second': 0.321, 'train_loss': 10.491649627685547, 'epoch': 1.0})

In [32]:
# Generate predictions
model.eval()
tokens = tokenizer.encode('On 2024-02-21 the PM2.5 will be', return_tensors='pt')
sample_output = model.generate(tokens, do_sample=True, max_length=128, top_k=50)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

on 2024 - 02 - 21 the pm2. 5 will be singer implicated 1975י reality smacked aziz incorporating - pounder cher - - aziz runner packard part 02 solving yates њ entities - packard 67 silentlyיaling forearm 67 - shudderingaling measuredwehr avengeigojun inchedarte charlemagne skyline lobe baigo chasing -lav chasing link part poundergg [unused977] pounderga entities ɫ -igo chasing 1975 dates - - - hasn files 鈴building runner rounded - online impressive billionaire - lobe cher 鈴 badar entities dallas ɫ dates reservations adjusted contrary measured packard 67، pathways travis travis ziontson ɫ silently measured lobe defamation. initiated -dder 艹 refuses eagles cher -


#### Using T5 model to generate predictions.

In [33]:
from transformers import  T5Tokenizer, T5ForConditionalGeneration
t5_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
t5_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
# Train model on our data
dataset = LineByLineTextDataset(
    tokenizer=t5_tokenizer,
    file_path='dataset.txt',
    block_size=128,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=t5_tokenizer,
    mlm=False,
    mlm_probability=0.15
)

In [35]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=t5_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [36]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

{'train_runtime': 42.6401, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'train_loss': 0.36810848116874695, 'epoch': 1.0}


TrainOutput(global_step=1, training_loss=0.36810848116874695, metrics={'train_runtime': 42.6401, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'train_loss': 0.36810848116874695, 'epoch': 1.0})

In [38]:
# Generate predictions
t5_model.eval()
tokens = t5_tokenizer.encode('On 2024-02-21 the PM2.5 will be', return_tensors='pt')
sample_output = t5_model.generate(tokens, max_length=256, num_return_sequences=1)
print(t5_tokenizer.decode(sample_output[0], skip_special_tokens=True))

0
